### USING OPENAI - (API EXPIRED)

In [24]:
import openai
from openai import OpenAI
import os
from dotenv import load_dotenv
import PyPDF2

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ''
        for page in reader.pages:
            text += page.extract_text()
    return text

load_dotenv()
OPENAI_API_KEY= os.getenv("OPENAI_API_KEY")

# Set your API key
openai.api_key = OPENAI_API_KEY

client = OpenAI(api_key=OPENAI_API_KEY)

def tailor_resume_with_openai(resume_text, jd_text):
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are an expert career consultant. Help tailor the resume to match the job description while keeping it realistic and professional."},
            {"role": "user", "content": f"Resume:\n{resume_text}\n\nJob Description:\n{jd_text}"}
        ],
        max_tokens=1000,
        temperature=0.8,
    )

    return completion.choices[0].message.content.strip()

from docx import Document

def generate_pdf(tailored_text, output_path):
    doc = Document()
    doc.add_paragraph(tailored_text)
    doc.save(output_path)

In [ ]:
# Extract text from resume and JD
resume_text = extract_text_from_pdf('/Users/karthiksagar/TailorMyCV/KarthikSagarN_Resume.pdf')
jd_text = extract_text_from_pdf('/Users/karthiksagar/TailorMyCV/jd.pdf')

# Tailor resume using GPT API
tailored_resume = tailor_resume_with_openai(resume_text, jd_text)

# Save tailored resume as a DOCX file
generate_pdf(tailored_resume, '/Users/karthiksagar/TailorMyCV/tailored_resume.docx')

### USING GROQ

In [50]:
import PyPDF2
from groq import Groq
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

# Function to extract text from PDF resume
def extract_text_from_resume(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ''
        for page in reader.pages:
            extracted_text = page.extract_text()
            if extracted_text:  # Check if extraction is successful
                text += extracted_text + "\n"
    return text.strip()

# Initialize Groq client
client = Groq(api_key=GROQ_API_KEY)

# Function to tailor resume
def tailor_resume_with_groq(resume_text, jd_text):
    prompt = f"""
You are an AI Resume Customization Specialist tasked with tailoring and optimizing a resume to align perfectly with a specific job role. Your goal is to produce a refined resume that maintains factual accuracy, preserves structure, and highlights the candidate's most relevant skills and achievements.

Here are the inputs for your task:

1. Existing Resume:
<existing_resume>
{resume_text}
</existing_resume>

2. Job Description:
<job_description>
{jd_text}
</job_description>

3. GitHub Profile & Project Information (if provided):
<github_info>
{{GITHUB_INFO}}
</github_info>

4. Personal Writeup (if provided):
<personal_writeup>
{{PERSONAL_WRITEUP}}
</personal_writeup>

Instructions:

1. Analyze all provided inputs thoroughly.

2. Wrap your analysis and tailoring strategy in <resume_analysis_and_strategy> tags:
   a. Extract and list key requirements from the job description.
   b. Identify matching skills and experiences from the existing resume.
   c. Note potential additions or modifications based on GitHub info and personal writeup (if provided).
   d. Create a mapping of job requirements to candidate qualifications.
   e. Outline your strategy for tailoring each section of the resume.
   f. Describe any structural changes you plan to make.
   g. Explain how you will emphasize related skills without fabricating experience. For example, if the job requires Python but the resume only lists Java, explain how you'll highlight transferable programming concepts.

3. Generate the tailored resume based on your analysis:
   a. Customize content to match the job description while maintaining the original structure.
   b. Ensure all dates and factual details remain accurate. Do not add false information or experience.
   c. Integrate relevant information from the GitHub profile/projects and personal writeup (if provided).
   d. Use a chronological format unless another format better serves the candidate's story.
   e. Keep the resume concise, ideally fitting within 1-2 pages.
   f. Develop a compelling summary or objective statement tailored to the job role.
   g. Use action verbs and quantify achievements to create impact.
   h. Prioritize and highlight skills, certifications, and experiences that match the job requirements.
   i. Ensure consistency throughout the resume in terms of style, dates, and factual information. 

4. DO NOT add experience that the candidate does not have. For example, if the job description asks about GPU workload management experience, but the input resume does not include experience in GPU management, DO NOT add experience that the candidate does not have.

5. Present the tailored resume in markdown format using <tailored_resume> tags.

Example output structure:

<resume_analysis_and_strategy>
[Your detailed analysis of the inputs and tailoring strategy]
</resume_analysis_and_strategy>

<tailored_resume>
# [Candidate Name]

[Contact Information]

## Summary
[Tailored summary or objective statement]

## Skills
[Relevant skills matching job requirements]

## Experience
### [Job Title] | [Company] | [Date Range]
- [Achievement 1]
- [Achievement 2]
- [Achievement 3]

[Additional experience sections as needed]

## Education
[Relevant education information]

## Projects
[Relevant projects, especially from GitHub if provided]

## Certifications
[Relevant certifications]
</tailored_resume>

Please proceed with your analysis and resume tailoring based on the provided inputs.
    """

    response = client.chat.completions.create(
        model="llama-3.3-70b-versatile",  # Verify the correct model name in GROQ's documentation
        messages=[{"role": "user", "content": prompt}],
        max_tokens=1000,
        temperature=0.8,
    )
    
    return response.choices[0].message.content


In [51]:
# Extract resume text
pdf_path = "/Users/karthiksagar/TailorMyCV/KarthikSagarN_Resume.pdf"  # Replace with the actual file path
text = extract_text_from_resume(pdf_path)

# Define job description text
jd_text = """
2026 Software Engineer Program - Full-Time - India

If you're passionate for technology to make a real difference, you'll find your place in our Software
Engineer Program.

Join an inspiring and curious team of technologists who will support you in developing new skills,
experiencing new projects and businesses, and taking your career where you want to go.
As a Software Engineer in the Software Engineer Program, you will start with an in-depth induction
program that teaches you about our businesses, builds on your knowledge of development
methodologies, and sharpens your professional skills. You will dive head-first into creating innovative
solutions that make a difference for our customers, clients and employees.
When you work at JPMorganChase, you're not just working at a global financial institution. You're an
integral part of one of the world's biggest tech companies. Across our global technology centres, our team
of more than 63,000 technologists design, build, deploy and run everything from enterprise technology
initiatives to big data and mobile solutions, as well as innovations in electronic payments, cybersecurity,
machine learning and cloud development. As a firm, we're partnering with FinTech and Silicon Valley tech
firms to deliver cutting-edge solutions to our clients and customers. Our $17 billion annual investment in
technology enables us to hire people to create innovative solutions that will not only transform the
financial services industry, but also change the world.

Job Responsibilities
• Create and improve the design, analytics, coding and testing for high-quality software and new
products.
• Own projects end-to-end, keeping teams and stakeholders up to speed on the progress of
what's being developed.
• Work and share ideas, information and innovation with our team of technologists from all over
the world.
• Develop through on-going training, mentorship and access to senior leaders.
• Participate in our Force for Good program to build real-world sustainable technology solutions
for social good organizations.

Required capabilities and skills
• Baseline knowledge of software, applications and technical processes within a given technical
discipline (e.g., cloud, artificial intelligence, mobile, etc.)
• Ability to work in large, collaborative teams to achieve organizational goals, with a passion to
build an inclusive and innovative culture

• Demonstrated ability to code in one or more general-purpose programming languages such as
Java, Python, Go & other programming languages or technologies like Robotics and Mobile Apps
etc.
• Understanding of software skills such as business analysis, development, maintenance, and
software improvement
• Proficiency in developmental toolsets
• Basic knowledge of industry-wide technology trends and best practices
• Exposure to agile methodologies such as CI/CD, Application Resiliency, and Security
• Familiarity with Big Data or Data Warehousing concepts
Required qualifications and eligibility criteria
• Pursuing a B.Tech/B.E degree with expected graduation by July 2026
• Streams and specializations considered are Computer Science/Information Science/ Information
Technology/Data Science/Artificial Intelligence/Big Data/IoT
• A well-rounded academic background with minimum (till date) CGPA of 8.5/10 or 85%
Preferred qualifications, capabilities, and skills
• Strong interpersonal and communication skills
• Ability to thrive in a fast-paced, collaborative environment
• Exceptional problem solving ability and fluency in English is essential
• Exposure to cloud technologies
• Experience with relational databases
"""

# Tailor the resume and print the output
tailored_resume = tailor_resume_with_groq(text, jd_text)

print(tailored_resume)

<resume_analysis_and_strategy>
The job description for the 2026 Software Engineer Program at JPMorganChase emphasizes the need for a strong foundation in software development, collaboration, and a passion for technology. Key requirements include:

* Baseline knowledge of software, applications, and technical processes
* Ability to work in large, collaborative teams
* Demonstrated ability to code in one or more general-purpose programming languages (e.g., Java, Python)
* Understanding of software skills such as business analysis, development, maintenance, and software improvement
* Proficiency in developmental toolsets
* Basic knowledge of industry-wide technology trends and best practices
* Exposure to agile methodologies and Big Data or Data Warehousing concepts

The existing resume highlights the candidate's experience in:

* Programming languages: Python, Java, HTML, CSS, JavaScript
* Development: Machine Learning, Deep Learning, NLP, Generative AI
* Databases/OS: MySQL, Linux, Wind

In [54]:
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas

def create_pdf(resume_text, output_path="/Users/karthiksagar/TailorMyCV/new_resume.pdf"):
    c = canvas.Canvas(output_path, pagesize=letter)
    c.setFont("Helvetica", 12)

    # Split text into lines for formatting
    lines = resume_text.split("\n")

    # Define starting position
    x, y = 50, 750

    for line in lines:
        if y < 50:  # Create a new page if space runs out
            c.showPage()
            c.setFont("Helvetica", 12)
            y = 750  # Reset Y position

        c.drawString(x, y, line)
        y -= 20  # Move to the next line

    c.save()
    print(f"PDF saved successfully as {output_path}")

In [55]:
create_pdf(tailored_resume)

PDF saved successfully as /Users/karthiksagar/TailorMyCV/new_resume.pdf
